In [1]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [2]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'oc1'  # euw1 na1 kr oc1
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196' # '12.12.450.4196' Version 12.12.448.6653 12.11.446.9344

In [3]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [4]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,DpOFwsnNPs01F0PGzBaZXx77dRVRpmEinXM5fYwk2K9d,7oIAu4T2gSCmvfNZRzISAYiDHJjCAirWgwsBkJppm8MllMw,0K48GAFGUcLoqA8K0JAoOkFUiNNkmfy3ebLt5d24rZmpil...,Typo Vampire,4075,1656494474000,99,DpOFwsnNPs01F0PGzBaZXx77dRVRpmEinXM5fYwk2K9d,Typo Vampire,587,I,155,106,False,False,True,True
1,zu74J2QwsNXXnU9ITZKtqzw3gbnxuFwGhNo38nZZCVIj,Q1uBipY4-_xKfrasWbtKEfkngMb0i4tDJTPNf_7zoYaYv3o,bNv1b-KBAJi6WTrnqjRDcTxnYBijLvDcIDaMTfrSGTm14j...,Kahdei,5389,1656493835728,376,zu74J2QwsNXXnU9ITZKtqzw3gbnxuFwGhNo38nZZCVIj,Kahdei,786,I,192,91,True,False,False,True
2,Eitk3JRXtwjp2oGYz33dpzlKpAa-4xA8tZoZuBVYIZDJ1g,csG3patLLddoTer4PRvUE2b4iMnIrfbcbvqQUWWKfCrfXjg,aY4uqTvEfizQcH1oR7Z4_dbS3PbqHODkkPluLTfox60C4V...,Colorful,936,1656471938000,180,Eitk3JRXtwjp2oGYz33dpzlKpAa-4xA8tZoZuBVYIZDJ1g,Colorful,619,I,269,166,True,False,False,False
3,kSSDwh3E9yXDXuw7xgpuj5CeTntJ6D75p2b7mBJ5qGuxOH4,_jXKI3EmrmlF8v6A4aNnohdSvUuutApBPN6zoNjumknt9o...,-7xzgRxivc0F44v07dTItabLAKy070nvTx_ttW9Mscgp2S...,WottS,4990,1656493030000,131,kSSDwh3E9yXDXuw7xgpuj5CeTntJ6D75p2b7mBJ5qGuxOH4,WottS,867,I,179,114,False,False,True,False
4,cAP-PDW31T8bNOhYrG6tSJLCLE3raXBcfFJyDIoZ9VNkEow,uQPBLdyPBDnQhfDMi6hYdu0PDRC_ySQw6OExLN5To7RsyL...,RX1PazO3ViQ9KLztmHGQ437gOvXd3ANBe9oihN8a0SJUOg...,Kjaos,29,1656432509000,1,cAP-PDW31T8bNOhYrG6tSJLCLE3raXBcfFJyDIoZ9VNkEow,Kjaos,715,I,127,61,False,False,False,False
5,fzKEcJ-8-ai-AJUNpELVB2jyX_eQkzmxJI_zUCLkT-4YMwY,8Wq9OVVYFY1oRkyz-djSbJjIwqqqX_O0JrpCncny00DT9g...,EAYrnw4XA-TiYNybbx9Gqof90Hz0ax-qjgFeVkViL6fYTG...,Anivia Kingdom,5067,1656492174000,12,fzKEcJ-8-ai-AJUNpELVB2jyX_eQkzmxJI_zUCLkT-4YMwY,Anivia Kingdom,735,I,149,73,False,False,True,True
6,KOD9iNO0yzXyUjCNAM6ETBWSEuj3jsCrPuYU4yWd5dG8NA,M-BGG4dQxtsBIqRrgz-prDFVUEHetBmWsvLRTTddkmrx2HU,QO-9u-w9o24dRLZREUh50KhLR7Icw1ybEnt9kFcguCRwvq...,Opal,1637,1656492174000,240,KOD9iNO0yzXyUjCNAM6ETBWSEuj3jsCrPuYU4yWd5dG8NA,Opal,744,I,133,61,False,False,True,False
7,ct6R3hxiIRMtJN8t8qme3qTlG7UPTXDddg-YxhFH8TNauA,P0xBaKD8VM2lSwN4GdUbZz-lUOJ7G22q10xuudsrS8v5V1A,9Yxs5hFtjolLkZgHuokMtZv_l2yy6rGJs-ALCk2HUu-4gw...,fread,588,1656492174000,249,ct6R3hxiIRMtJN8t8qme3qTlG7UPTXDddg-YxhFH8TNauA,fread,511,I,104,50,False,False,True,False
8,_rmxuAUQ1ZNhFP7TvX8OHSpA0g7_Gf3wQwf7GVMwDof6O4...,mHGdkK987jKUJBg3KoVc2ca5a0jISDX1V-9uQ5nEvLG08b...,AJ-S7vbOwZOUKMa4Iz9ZdffnGmgDnDqQqPXX40p6bq5DSh...,jilhf,29,1656486929000,1,_rmxuAUQ1ZNhFP7TvX8OHSpA0g7_Gf3wQwf7GVMwDof6O4...,jilhf,549,I,142,93,False,False,True,False
9,H79RhtkoeHBWV_fT0OleEz570Un-6H_DKFW9-BZTGhdRA8E,Bm9sSal__SlnEBXTcFtaOSv0_2kUHdzo4YrxAaXF_CBxpf...,BuPGnWoyn4oDOgWiLBH3aa-HZ_M2n3MCjhtdzuIR2ITfo9...,Dawnglade,4409,1656242384000,31,H79RhtkoeHBWV_fT0OleEz570Un-6H_DKFW9-BZTGhdRA8E,Dawnglade,803,I,120,53,False,False,False,True


In [5]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [6]:
len(uniq_matches_id)

533

In [7]:
len(matches_asset)

681

In [8]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [9]:
len(uniq_matches)

533

In [10]:
matches_asset[300]['info']['game_version']

'Version 12.12.450.4196 (Jun 24 2022/13:37:31) [PUBLIC] <Releases/12.12>'

In [11]:
latest_matches = [match for match in uniq_matches if LATEST_RELEASE in match['info']['game_version']]

In [12]:
len(latest_matches)

381

In [13]:
# latest_matches[0]['info']

In [14]:
matches_array = []

for match_row in latest_matches:
    match_id = match_row['metadata']['match_id']

    for participant in match_row['info']['participants']:
        match = {}
        match['match_id'] = match_id
        # match['level'] = participant['level']
        match['placement'] = participant['placement']
        # match['players_eliminated'] = participant['players_eliminated']
        # match['total_damage_to_players'] = participant['total_damage_to_players']

        for augment_index, augment in enumerate(participant['augments']):
            if augment == 'TFT7_Augment_GuildLootHR':
                augment = 'TFT7_Augment_BandOfThieves1'
            match[f'augment{augment_index}'] = augment

        for trait_index, trait in enumerate(participant['traits']):
            match[f'{trait["name"]}'] = trait["tier_current"]

        for unit_index, unit in enumerate(participant['units']):
            match[f'{unit["character_id"]}'] = unit["tier"]
            for item_index, item in enumerate(unit['itemNames']):
                match[f'{unit["character_id"]}_item{item_index}'] = item

        matches_array.append(match)

In [15]:
matches_array[0]
# len(matches_array)

{'match_id': 'OC1_526869482',
 'placement': 4,
 'augment0': 'TFT6_Augment_Electrocharge1',
 'augment1': 'TFT7_Augment_BandOfThieves1',
 'augment2': 'TFT6_Augment_ComponentGrabBag',
 'Set7_Assassin': 1,
 'Set7_Bard': 1,
 'Set7_Bruiser': 1,
 'Set7_Cavalier': 1,
 'Set7_Dragon': 1,
 'Set7_Guardian': 0,
 'Set7_Guild': 2,
 'Set7_Legend': 0,
 'Set7_Mystic': 0,
 'Set7_Ragewing': 1,
 'Set7_Shimmerscale': 1,
 'Set7_Swiftshot': 0,
 'Set7_Tempest': 1,
 'Set7_Warrior': 0,
 'TFT7_Qiyana': 1,
 'TFT7_Shen': 2,
 'TFT7_Hecarim': 2,
 'TFT7_Hecarim_item0': 'TFT_Item_ThiefsGloves',
 'TFT7_Hecarim_item1': 'TFT_Item_LocketOfTheIronSolari',
 'TFT7_Hecarim_item2': 'TFT_Item_GargoyleStoneplate',
 'TFT7_Talon': 1,
 'TFT7_Talon_item0': 'TFT7_Item_CavalierEmblemItem',
 'TFT7_Talon_item1': 'TFT_Item_ZekesHerald',
 'TFT7_Talon_item2': 'TFT_Item_PowerGauntlet',
 'TFT7_Xayah': 2,
 'TFT7_Xayah_item0': 'TFT_Item_GuinsoosRageblade',
 'TFT7_Xayah_item1': 'TFT_Item_LastWhisper',
 'TFT7_Xayah_item2': 'TFT_Item_RunaansHurric

In [16]:
matches_league_df = pd.json_normalize(matches_array)

In [17]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bard,Set7_Bruiser,Set7_Cavalier,Set7_Dragon,...,TFT7_Sejuani_item2,TFT7_Skarner_item2,TFT7_Aatrox,TFT7_Thresh_item1,TFT7_Thresh_item2,TFT7_Aatrox_item0,TFT7_Aatrox_item1,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Aatrox_item2
0,OC1_526869482,4,TFT6_Augment_Electrocharge1,TFT7_Augment_BandOfThieves1,TFT6_Augment_ComponentGrabBag,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OC1_526869482,8,TFT7_Augment_RevelTrait,TFT6_Augment_PandorasItems,TFT7_Augment_RevelPartyTime,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OC1_526869482,5,TFT6_Augment_CyberneticShell1,TFT6_Augment_TinyTitans,TFT7_Augment_UrfsGrabBag1,2.0,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OC1_526869482,2,TFT6_Augment_SecondWind1,TFT6_Augment_FirstAidKit,TFT6_Augment_SecondWind2,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OC1_526869482,7,TFT6_Augment_TinyTitans,TFT6_Augment_MakeshiftArmor1,TFT6_Augment_TargetDummies,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3043,OC1_526747235,2,TFT7_Augment_BruiserTitanicStrength,TFT6_Augment_SecondWind1,TFT7_Augment_WarriorTiamat,NaN,NaN,3.0,NaN,1.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3044,OC1_526747235,1,TFT6_Augment_SunfireBoard,TFT7_Augment_BandOfThieves1,TFT6_Augment_FuturePeepers,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3045,OC1_526747235,6,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_SecondWind1,TFT6_Augment_SunfireBoard,NaN,NaN,0.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3046,OC1_526747235,5,TFT6_Augment_CelestialBlessing2,TFT7_Augment_ScalescornTrait,TFT7_Augment_ScalescornEmblem,3.0,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Assassin,Set7_Bard,Set7_Bruiser,Set7_Cavalier,Set7_Dragon,...,TFT7_Sejuani_item2,TFT7_Skarner_item2,TFT7_Aatrox,TFT7_Thresh_item1,TFT7_Thresh_item2,TFT7_Aatrox_item0,TFT7_Aatrox_item1,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Aatrox_item2
0,OC1_526869482,4,TFT6_Augment_Electrocharge1,TFT7_Augment_BandOfThieves1,TFT6_Augment_ComponentGrabBag,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,OC1_526869482,8,TFT7_Augment_RevelTrait,TFT6_Augment_PandorasItems,TFT7_Augment_RevelPartyTime,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OC1_526869482,5,TFT6_Augment_CyberneticShell1,TFT6_Augment_TinyTitans,TFT7_Augment_UrfsGrabBag1,2.0,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OC1_526869482,2,TFT6_Augment_SecondWind1,TFT6_Augment_FirstAidKit,TFT6_Augment_SecondWind2,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OC1_526869482,7,TFT6_Augment_TinyTitans,TFT6_Augment_MakeshiftArmor1,TFT6_Augment_TargetDummies,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3043,OC1_526747235,2,TFT7_Augment_BruiserTitanicStrength,TFT6_Augment_SecondWind1,TFT7_Augment_WarriorTiamat,NaN,NaN,3.0,NaN,1.0,...,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3044,OC1_526747235,1,TFT6_Augment_SunfireBoard,TFT7_Augment_BandOfThieves1,TFT6_Augment_FuturePeepers,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3045,OC1_526747235,6,TFT7_Augment_UrfsGrabBag1,TFT6_Augment_SecondWind1,TFT6_Augment_SunfireBoard,NaN,NaN,0.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3046,OC1_526747235,5,TFT6_Augment_CelestialBlessing2,TFT7_Augment_ScalescornTrait,TFT7_Augment_ScalescornEmblem,3.0,NaN,1.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.pickle'))
matches_league_df.to_csv(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_{LATEST_RELEASE}_matches.csv'), index=False)